# RENDER

#### ! (demo figure) render env

In [ ]:
import experiments.db3.db as db

import matplotlib.pyplot as plt

In [ ]:
env=db.envF(
    alias='test',
    testing=True,
    reward_scheme=dict(
            dis_target_point=           1.0,
            dis_neighbour=              0.0,
            hull_formed=                0.0,
            dis_target_radius=          1.0,
            all_unsafe=                 1.0,
            all_neighbour=              1.0,
            occluded_neighbour=         1.0,
        ) ,
    record_state_history=False, 
    initial_states=db.isdF(names='A', target_point=(70,70), target_radius=2.0, reset_noise=0.0, validation=False)
)

In [ ]:
env.reset()
fig=env.render(local_sensors=False, reward_signal=False, fill_bots=False, show_com=2)
plt.show()
fig.savefig('A.png')

In [ ]:
s=db.isdF(names='B', target_point=(7,-7), target_radius=2.0, reset_noise=0.0, validation=False)[0]
s.points[1]=(100,100)
env=db.envF(
    alias='test',
    testing=True,
    reward_scheme=dict(
            dis_target_point=           1.0,
            dis_neighbour=              0.0,
            hull_formed=                0.0,
            dis_target_radius=          1.0,
            all_unsafe=                 1.0,
            all_neighbour=              1.0,
            occluded_neighbour=         1.0,
        ) ,
    record_state_history=False, 
    initial_states=[s]
)

In [ ]:
env.reset()
fig=env.render(local_sensors=False, reward_signal=False, fill_bots=False, show_com=2)
plt.show()
fig.savefig('B.png')

# render select

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
results_dir = '__results__'
#os.makedirs(results_dir, exist_ok=True)
os.chdir(results_dir)

#import argparse
import importlib
from fatbot import FAKE

dbs = ['db6', 'db8', 'db10']#, 

exper = [
    ('D', 'CD'),
    ('P', 'CP'),
    ('S', 'CS'),
    ('D', 'CC'),
    ('P', 'CC'),
    ('S', 'CC'),
]


# render n_STEPS

In [ ]:
episodes = 20

In [ ]:
for db in dbs:
    kalias = {}
    for sd,cc in exper:
        argc =  FAKE(
            db=     db,
            cs=     'config',
            cc=     cc,
            sd=     sd)

        #runs = [bool(int(i)) for i in argc.run]

        cs = importlib.import_module(f'experiments.{argc.db}.{argc.cs}')
        cc = getattr(cs, argc.cc)
        alias = f'{argc.db}_{argc.cs}_{argc.cc}_{argc.sd}'
        module = cc(alias=alias, common_initial_states=argc.sd)

        #module.do_final_test()

        #print('alias')
        
        average_return, avg_steps, sehist, tehist, test_history, fi, ff, switch_history=\
            module.do_final_test_(episodes=episodes)
        print(len(tehist)==len(switch_history))
        if (len(tehist)==len(switch_history)):
            #print(f'...adding {alias} :: {len(tehist)}=={len(switch_history)}')
            ka = []
            for t,s in zip(tehist, switch_history): ka.append([s, len(t)-s, len(t) ])
            ka = np.array(ka)
            kalias[alias]=ka
            #print(f'...adding {alias} :: {ka.shape}::{ka}')
        else:
            print(f'...skipping {alias} :: {len(tehist)}!={len(switch_history)}')
            #kalias[alias]=None
    # for k,v in kalias.items():
    #     fig, ax=plt.subplots(3, 1, figsize=(12,8))
        
    #     for i in range(3):
    #         ax[i].plot(v[:, i])
    #         ax[i].set_ylim(bottom=0)
            

    #     plt.show()

    jalias = {}
    for k,v in kalias.items():
        jalias[k] = np.mean(v, axis=0)

    np.savez(f'{db}_{episodes}.npz', **kalias)


In [ ]:
for db in dbs:
    npz=np.load(f'{db}_{episodes}.npz')
    for k in npz.keys():
        print(k, np.mean(npz[k],axis=0))
    npz.close()
    # results 2

In [ ]:
"""
db6_config_CD_D [47.9 47.  94.9]
db6_config_CP_P [67.55 32.   99.55]
db6_config_CS_S [ 50.7   51.75 102.45]
db6_config_CC_D [ 53.7   46.55 100.25]
db6_config_CC_P [ 34.8  77.8 112.6]
db6_config_CC_S [39.75 36.85 76.6 ]
db8_config_CD_D [ 56.75  61.3  118.05]
db8_config_CP_P [59.7 29.8 89.5]
db8_config_CS_S [ 51.25  62.65 113.9 ]
db8_config_CC_D [ 80.35  64.85 145.2 ]
db8_config_CC_P [ 46.6   59.45 106.05]
db8_config_CC_S [ 45.  112.5 157.5]
db10_config_CD_D [ 73.1   96.45 169.55]
db10_config_CP_P [ 60.2   82.35 142.55]
db10_config_CS_S [ 67.3  96.1 163.4]
db10_config_CC_D [ 79.7   78.95 158.65]
db10_config_CC_P [ 44.05  91.   135.05]
db10_config_CC_S [ 65.05  84.55 149.6 ]
"""

# render animation

In [ ]:
db = dbs[1]
exper = [
    #('D', 'CD'),
    #('P', 'CP'),
    #('S', 'CS'),
    ('D', 'CC'),
    ('P', 'CC'),
    ('S', 'CC'),
]
for sd,cc in exper:
    argc =  FAKE(
        db=     db,
        cs=     'config',
        cc=     cc,
        sd=     sd)

    cs = importlib.import_module(f'experiments.{argc.db}.{argc.cs}')
    cc = getattr(cs, argc.cc)
    alias = f'{argc.db}_{argc.cs}_{argc.cc}_{argc.sd}'
    module = cc(alias=alias, common_initial_states=argc.sd)
    print(f'{alias=}')
    module.do_final_test()

    
